In [1]:
from pyzx import *
from fractions import Fraction

In [2]:
class SQW:
    def __init__(self):
        self.qubit_amount = 3
        self.circ = Circuit(qubit_amount=self.qubit_amount)
    def setup(self,init=2):
        self.circ.add_gate("NOT",init)
    def add_layer(self, n=1):
        for i in range(n):
            self.circ.add_gate("XPhase",0 , phase=Fraction(2,3))
            self.circ.add_gate("TOF",0,1,2)
            self.circ.add_gate("CNOT",0,1)
            self.circ.add_gate("NOT",0)
            
            self.circ.add_gate("XPhase",0 , phase=Fraction(2,3))
            
            self.circ.add_gate("NOT",0)
            self.circ.add_gate("NOT",1)
            self.circ.add_gate("TOF",0,1,2)
            self.circ.add_gate("NOT",1)
            self.circ.add_gate("CNOT",0,1)

In [3]:
two_step = SQW()
two_step.setup()
two_step.add_layer(1)

In [4]:
g = two_step.circ
print("With syntactic sugar")
draw(g)
#settings.tikzit_location = "/usr/bin/tikzit"
#tikz.tikzit(fr_opt)
print("Without syntactic sugar")
draw(g.to_graph())
print(extract_simple(g.to_graph()).stats())

With syntactic sugar


Without syntactic sugar


Circuit  on 3 qubits with 39 gates.
        16 is the T-count
        23 Cliffords among which
        14 2-qubit gates (14 CNOT, 0 other) and
        4 Hadamard gates.


# Hand picked simplifications
The simplifications below where chosen because of the structure of the diagram above.

- phase_free_simp -> spider_simp followed by a bialg_simp
- id_simp to remove any $n*2\pi$ rotations
- spider_simp to fuse spiders after the id
- id_simp to remove any $n*2\pi$ rotations

In [5]:
def qw_opt(g):
    phase_free_simp(g,quiet=False)
    id_simp(g,quiet=False)
    spider_simp(g, quiet=False)
    id_simp(g,quiet=False)
    #supplementarity_simp(g,quiet=False)
    g.normalize()

In [6]:
opt = g.to_graph()
qw_opt(opt)
draw(opt)

spider_simp: 7. 6. 4. 2. 1.  5 iterations
id_simp: 3. 1.  2 iterations


In [7]:
result = extract_simple(opt)
print(result.stats(),"\nEQUAL: " , g.verify_equality(result))
print("Result:")
draw(result)
#settings.tikzit_location = "/usr/bin/tikzit"
#tikz.tikzit(result)
print("Original:")
draw(g.to_graph())
print(extract_simple(g.to_graph()).stats())

Circuit  on 3 qubits with 31 gates.
        16 is the T-count
        15 Cliffords among which
        10 2-qubit gates (10 CNOT, 0 other) and
        2 Hadamard gates. 
EQUAL:  True
Result:


Original:


Circuit  on 3 qubits with 39 gates.
        16 is the T-count
        23 Cliffords among which
        14 2-qubit gates (14 CNOT, 0 other) and
        4 Hadamard gates.


# Full Reduce
This simplification is useful to reduce the T-count of a circuit due to the phase gadget optimizations implemented in the algorithm

In [8]:
fr = g.to_graph()
full_reduce(fr, quiet=False)
fr.normalize()
draw(fr)
#settings.tikzit_location = "/usr/bin/tikzit"
#tikz.tikzit(fr)

spider_simp: 7. 6. 4. 2. 1.  5 iterations
id_simp: 3. 1.  2 iterations
pivot_simp: 1.  1 iterations
pivot_gadget_simp: 6. 2. 2.  3 iterations
id_simp: 2.  1 iterations
spider_simp: 1.  1 iterations
gadget_simp: 4.  1 iterations
lcomp_simp: 2. 2.  2 iterations


In [9]:
fr_opt = extract_circuit(fr.copy())
print(fr_opt.stats(),"\nEQUAL: " , g.verify_equality(fr_opt))
draw(fr_opt)
#settings.tikzit_location = "/usr/bin/tikzit"
#tikz.tikzit(fr_opt)

Circuit  on 3 qubits with 48 gates.
        10 is the T-count
        38 Cliffords among which
        18 2-qubit gates (5 CNOT, 13 other) and
        20 Hadamard gates. 
EQUAL:  True


PyZX tends to perform better with a lot of hadamard gates, although this circuit could use some optimizations

We can remove some hadamards by some colour changes and the total number of gates with a few spider simps and id simps

In [10]:
fr_graph = fr_opt.to_graph()
# Simple simplifications
spider_simp(fr_graph,quiet=False)
id_simp(fr_graph,quiet=False)
to_rg(fr_graph)
#The simps above dont change the graph too much so we can use the extract_simple
fr_opt = extract_simple(fr_graph)
print()
print(fr_opt.stats(),"\nEQUAL: " , g.verify_equality(fr_opt))
draw(fr_opt)
draw(fr)

spider_simp: 13. 10. 8. 2. 2. 1. 1.  7 iterations
id_simp: 2.  1 iterations

Circuit  on 3 qubits with 34 gates.
        10 is the T-count
        24 Cliffords among which
        20 2-qubit gates (15 CNOT, 5 other) and
        4 Hadamard gates. 
EQUAL:  True


In [11]:
#settings.tikzit_location = "/usr/bin/tikzit"
#tikz.tikzit(fr_opt)

In [12]:
indexes = list(filter(lambda x: isinstance(fr_opt.to_graph().phases()[x],Fraction),fr_opt.to_graph().phases())) 

In [13]:
rotations = list(map(lambda x: fr_opt.to_graph().phases()[x], indexes))

In [14]:
useful = rotations[4:]

for i in range(len(useful)):
    print(str(useful[i]) + " " , end="")
    if (i+1) % 6 == 0:
        print()


len(rotations)//6, len(rotations)

1/4 2/3 7/4 1/4 2/3 7/4 
1/4 2/3 7/4 1/4 5/3 7/4 
1/4 5/3 7/4 1/4 2/3 1/4 
7/4 5/3 1/4 7/4 5/3 1/4 
7/4 5/3 1/4 7/4 5/3 7/4 
1/4 2/3 1/4 7/4 2/3 7/4 
1/4 2/3 7/4 1/4 2/3 7/4 
1/4 2/3 7/4 1/4 7/4 1/2 
1 1/4 

(9, 54)

#1/4 2/3 7/4 1/4 2/3 7/4 
#1/4 2/3 7/4 1/4 5/3 7/4 
#1/4 5/3 7/4 1/4 2/3 1/4 
#7/4 5/3 1/4 7/4 5/3 1/4 
#7/4 5/3 1/4 7/4 5/3 7/4 
#1/4 2/3 1/4 7/4 2/3 7/4 
#1/4 2/3 7/4 1/4 2/3 7/4 
#1/4 2/3 7/4 1/4 5/3 1/4 
#7/4 5/3 7/4 1/4 5/3 1/4 
#7/4 2/3 1/4 7/4 2/3 1/4 
#7/4 5/3 7/4 1/4 2/3 7/4 
#1/4 2/3 7/4 1/4 5/3 7/4 
#1/4 5/3 1/4 7/4 2/3 1/4 
#7/4 2/3 1/4 7/4 2/3 1/4 
#7/4 2/3 1/4 7/4 2/3 7/4 
#1/4 5/3 7/4 1/4 2/3 1/4 
#7/4 5/3 1/4 7/4 2/3 7/4 
#1/4 2/3 1/4 7/4 2/3 1/4 
#7/4 2/3 7/4 1/4 2/3 7/4 
#1/4 2/3 1/4 7/4 5/3 7/4 
#1/4 5/3 1/4 7/4 5/3 7/4 
#1/4 5/3 1/4 7/4 7/4 1/2 
#1/4 

There seems to be a pattern here, although it's not that well defined

In [15]:
print(fr_opt.to_qasm())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[3];
h q[1];
rx(0.25*pi) q[1];
cx q[2], q[1];
rx(0.75*pi) q[2];
rx(0.6666666666666666*pi) q[0];
h q[0];
cx q[0], q[2];
h q[0];
h q[0];
cx q[0], q[1];
h q[0];
rz(1.75*pi) q[0];
cx q[0], q[2];
cx q[0], q[1];
h q[0];
h q[1];
cx q[1], q[2];
h q[1];
cx q[0], q[2];
cx q[2], q[1];
cx q[0], q[1];
h q[1];
rx(0.25*pi) q[0];
h q[0];
cx q[0], q[1];
h q[0];
rz(0.6666666666666666*pi) q[0];
cx q[0], q[2];
rx(1.75*pi) q[0];
cx q[0], q[2];
rx(0.25*pi) q[0];
h q[0];
cx q[0], q[1];
h q[0];
rz(0.6666666666666666*pi) q[0];
cx q[0], q[2];
rx(1.75*pi) q[0];
h q[0];
cx q[0], q[2];
h q[0];
cx q[0], q[2];
rx(0.25*pi) q[0];
h q[0];
cx q[0], q[1];
h q[0];
rz(0.6666666666666666*pi) q[0];
cx q[0], q[2];
rx(1.75*pi) q[0];
cx q[0], q[2];
rx(0.25*pi) q[0];
h q[0];
cx q[0], q[1];
h q[0];
rz(1.6666666666666667*pi) q[0];
rx(1.75*pi) q[0];
h q[0];
cx q[0], q[2];
h q[0];
cx q[0], q[2];
rx(0.25*pi) q[0];
h q[0];
cx q[0], q[1];
h q[0];
rz(1.6666666666666667*pi) q[0];
rx(1.75*pi) q[0]